In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error

In [ ]:
# mesh size 5 or 6
_MESH_SIZE = 6

# 45e300_on6_precip vs 56e300_on6_precip
# 45e300_on6_temp   vs 56e300_on6_temp
# 45e1k_on6_precip  vs 56e1k_on6_precip
# 45e1k_on6_temp    vs 56e1k_on6_temp
# 46e300_on6_precip vs 56e300_on6_precip
# 46e300_on6_temp   vs 56e300_on6_temp
# 46e1k_on6_precip  vs 56e1k_on6_precip
# 46e1k_on6_temp    vs 56e1k_on6_temp
predlist = [
  ['45e300_on6_precip', '56e300_on6_precip', '45e300_on6_temp', '56e300_on6_temp'],
  ['45e1k_on6_precip',  '56e1k_on6_precip',  '45e1k_on6_temp',  '56e1k_on6_temp'],
  # ['46e300_on6_precip', '56e300_on6_precip', '46e300_on6_temp', '56e300_on6_temp'], # comparison has no meaning
  # ['46e1k_on6_precip',  '56e1k_on6_precip',  '46e1k_on6_temp',  '56e1k_on6_temp']
]
_IDX_A = 1
# _IDX_B = 3

# Load the predictions from the .npy file
pred_a = [np.load(f'results/{predlist[_IDX_A][i]}.npy') for i in range(4)]
# pred_b = [np.load(f'results/{predlist[_IDX_B][i]}.npy') for i in range(4)]

# Load the data from the .npz file
gc = np.load(f'dataset/targets/data_m{_MESH_SIZE}.npz')

# Access target in the .npz file
data_arrays = {key: gc[key] for key in gc.files}
target = data_arrays['target']

# Calculate Mean Squared Error to 3 decimal places
mse_a = [f"{mean_squared_error(target, pred_a[i]):.3f}" for i in range(4)]
# mse_b = [f"{mean_squared_error(target, pred_b[i]):.3f}" for i in range(4)]

for i in range(4):
  print(f"Mean Squared Error {predlist[_IDX_A][i]}: {mse_a[i]}")
  # print(f"Mean Squared Error {predlist[_IDX_B][i]}: {mse_b[i]}")


# Update to real data variable names
real_predlist = [[item.replace('precip', 'total_precipitation_6hr') for item in sublist] for sublist in predlist]
real_predlist = [[item.replace('temp', 'temperature') for item in sublist] for sublist in real_predlist]
preds = real_predlist[_IDX_A] #+ real_predlist[_IDX_B]

df = pd.DataFrame({
  '': ['Prediction', 'Train Result'] * 2,
  'Input Mesh Size': [pred.split('_')[0][0] for pred in preds],
  'Output Mesh Size': [pred.split('_')[0][1] for pred in preds],
  'Data Variable': ['_'.join(pred.split('_')[2:]) for pred in preds],
  'Mean Squared Error': mse_a #+ mse_b
})
epochs = predlist[_IDX_A][0].split('_')[0][3:]
pred_mesh_size = predlist[_IDX_A][0].split('_')[1][2]

fig, ax = plt.subplots()
fig.set_size_inches(12, 4)
ax.set_title('MSE Comparison',fontsize=16)
ax.text(0, 0.04, f'All Model Epochs = {epochs}\nAll Predictions on Mesh Size {pred_mesh_size}', horizontalalignment='center', fontsize=9)
ax.axis('off')
ax.axis('tight')
ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
plt.savefig(f"results/mse_{_IDX_A}.png")
plt.show()